In [1]:
import keras
import pandas
import sklearn
import scipy
import dask.dataframe as dd
import numpy

C:\Users\danil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# use this code to transfer between csv to parquet
df.to_parquet("train_data.parquet")

In [ ]:
training_data = pandas.read_csv("train_data.csv")

In [ ]:
training_sample = training_data.sample(frac=0.3)

In [ ]:
training_sample.shape

In [ ]:
training_sample.to_csv("train_data_sample.csv", index=False)

# read sample if already available

In [2]:
training_data = pandas.read_csv("train_data_sample.csv")

In [4]:
training_data.columns

Index(['id', 'UF_1', 'UF_2', 'UF_3', 'UF_4', 'UF_5', 'UF_6', 'UF_7', 'IDADE',
       'SEXO_1',
       ...
       'CEP4_7', 'CEP4_8', 'CEP4_9', 'CEP4_10', 'CEP4_11', 'CEP4_12',
       'CEP4_13', 'CEP4_14', 'IND_BOM_1_1', 'IND_BOM_1_2'],
      dtype='object', length=246)

In [5]:
# keep ids and their index on database for further reference
ids = training_data["id"]

In [6]:
features = training_data.drop(["IND_BOM_1_1", "IND_BOM_1_2", "id"], axis=1)

In [7]:
features.head(10)

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_5,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14
0,1,1,1,0,0,0,0,0.217846,0,0.111111,...,0,1,0,0,1,1,0,0,0,0
1,0,0,1,1,0,1,0,0.750400,0,0.111111,...,0,0,0,1,0,1,1,0,0,0
2,1,0,0,0,1,1,0,0.074953,0,0.111111,...,0,0,0,0,0,0,0,1,1,1
3,1,1,1,0,0,0,0,0.355855,0,0.111111,...,0,0,0,1,1,1,0,0,0,1
4,1,0,1,0,0,1,0,0.930834,1,0.111111,...,1,1,0,1,0,0,0,1,0,0
5,1,1,1,0,0,0,0,0.678045,0,0.111111,...,1,0,1,0,0,1,0,0,1,0
6,1,1,0,0,0,0,1,0.485231,1,0.111111,...,0,1,1,0,1,0,1,0,0,0
7,0,1,1,0,1,0,0,0.654419,1,0.111111,...,0,1,0,0,0,0,0,1,1,1
8,1,1,0,1,0,0,0,0.358808,1,0.111111,...,1,1,0,0,0,0,1,1,0,1
9,1,1,1,0,0,0,0,0.132485,1,0.111111,...,0,0,1,1,1,1,0,0,0,1


In [8]:
labels = training_data["IND_BOM_1_1"]

Vou tentar reduzir a dimensionalidade do dataframe utilizando LDA para poder analisar 
melhor um subconjunto de variáveis e mensurar a acurácia

## Data Cleaning

In [9]:
training_data.dtypes.value_counts()

float64    144
int64      102
dtype: int64

Pandas leu muito dos valores de forma errada, gerando até problemas para o uso de memória, existem formas
melhores de representar estas variáveis.

In [10]:
training_data.columns

Index(['id', 'UF_1', 'UF_2', 'UF_3', 'UF_4', 'UF_5', 'UF_6', 'UF_7', 'IDADE',
       'SEXO_1',
       ...
       'CEP4_7', 'CEP4_8', 'CEP4_9', 'CEP4_10', 'CEP4_11', 'CEP4_12',
       'CEP4_13', 'CEP4_14', 'IND_BOM_1_1', 'IND_BOM_1_2'],
      dtype='object', length=246)

In [11]:
training_data.dtypes

id                                  int64
UF_1                                int64
UF_2                                int64
UF_3                                int64
UF_4                                int64
UF_5                                int64
UF_6                                int64
UF_7                                int64
IDADE                             float64
SEXO_1                              int64
NIVEL_RELACIONAMENTO_CREDITO01    float64
NIVEL_RELACIONAMENTO_CREDITO02    float64
BANCO_REST_IRPF_ULTIMA_1            int64
BANCO_REST_IRPF_ULTIMA_2            int64
BANCO_REST_IRPF_ULTIMA_3            int64
BANCO_REST_IRPF_ULTIMA_4            int64
BANCO_REST_IRPF_ULTIMA_5            int64
BANCO_REST_IRPF_ULTIMA_6            int64
BANCO_REST_IRPF_ULTIMA_7            int64
ATIVIDADE_EMAIL                   float64
EXPOSICAO_ENDERECO                float64
EXPOSICAO_EMAIL                   float64
EXPOSICAO_TELEFONE                float64
ATIVIDADE_ENDERECO                

In [12]:
category_columns = ["UF_1", "UF_2", "UF_3", "UF_4", "UF_5", "UF_6", "UF_7",
                   "BANCO_REST_IRPF_ULTIMA_1", "BANCO_REST_IRPF_ULTIMA_2", "BANCO_REST_IRPF_ULTIMA_3",
                   "BANCO_REST_IRPF_ULTIMA_4", "BANCO_REST_IRPF_ULTIMA_5", "BANCO_REST_IRPF_ULTIMA_6",
                   "BANCO_REST_IRPF_ULTIMA_7", "FLAG_BOLSA_FAMILIA_1", "SIGLA_PARTIDO_FILIADO_1",
                   "SIGLA_PARTIDO_FILIADO_2", "SIGLA_PARTIDO_FILIADO_3", "SIGLA_PARTIDO_FILIADO_4",
                   "SIGLA_PARTIDO_FILIADO_5", "SIGLA_PARTIDO_FILIADO_6", "SIGLA_PARTIDO_FILIADO_7",
                   "FLAG_FILIADO_PARTIDO_POLITICO_1", "FLAG_PROUNI_1", "RENDA_VIZINHANCA_1", 
                   "RENDA_VIZINHANCA_2", "RENDA_VIZINHANCA_3", "RENDA_VIZINHANCA_4", 
                    "COMPARATIVO_RENDA_CEP_1", "COMPARATIVO_RENDA_CEP_2", "COMPARATIVO_RENDA_CEP_3",
                   "COMPARATIVO_RENDA_CEP_4", "COMPARATIVO_RENDA_CEP_5", "CLASSE_SOCIAL_CONSUMIDOR_1",
                   "CLASSE_SOCIAL_CONSUMIDOR_2", "CLASSE_SOCIAL_CONSUMIDOR_3", "CLASSE_SOCIAL_CONSUMIDOR_4",
                   "FLAG_REDE_SOCIAL_1", "FLAG_REDE_SOCIAL_2", "FLAG_REDE_SOCIAL_3",
                   "CEP1_1", "CEP1_2", "CEP1_3", "CEP1_4", "CEP1_5", "CEP2_1", "CEP2_2", "CEP2_3", "CEP2_4",
                   "CEP2_5", "CEP2_6", "CEP2_7", "CEP2_8", "CEP2_9", "CEP3_1", "CEP3_2", "CEP3_3", "CEP3_4",
                   "CEP3_5", "CEP3_6", "CEP3_7", "CEP3_8", "CEP3_9", "CEP3_10", "CEP3_11", "CEP3_12",
                   "CEP4_1", "CEP4_2", "CEP4_3", "CEP4_4", "CEP4_5", "CEP4_6", "CEP4_7", "CEP4_8", "CEP4_9",
                   "CEP4_10", "CEP4_11", "CEP4_12", "CEP4_13", "CEP4_14"]

ordered_category_columns = ["NIVEL_RELACIONAMENTO_CREDITO02", "EXPOSICAO_CONSUMIDOR_EMAILS", 
                            "EXPOSICAO_CONSUMIDOR_TELEFONES"]

In [13]:
for column in training_data.columns:
    print(column)
    values = training_data[column].value_counts()
    if type(values) == list:
        print(values[:10])
    else:
        print(values.head(10))
    print(len(values))
    

id
4094      1
270713    1
210764    1
206670    1
112591    1
259924    1
261973    1
56484     1
336343    1
108493    1
Name: id, dtype: int64
116759
UF_1
1    103790
0     12969
Name: UF_1, dtype: int64
2
UF_2
1    80712
0    36047
Name: UF_2, dtype: int64
2
UF_3
0    61114
1    55645
Name: UF_3, dtype: int64
2
UF_4
0    82157
1    34602
Name: UF_4, dtype: int64
2
UF_5
0    88506
1    28253
Name: UF_5, dtype: int64
2
UF_6
0    91341
1    25418
Name: UF_6, dtype: int64
2
UF_7
0    94902
1    21857
Name: UF_7, dtype: int64
2
IDADE
5.506237e-16    1273
1.000000e+00    1158
3.078642e-01      36
2.940066e-01      34
3.008786e-01      32
3.130324e-01      29
2.189822e-01      24
3.908966e-01      21
3.636924e-01      21
5.574156e-01      21
Name: IDADE, dtype: int64
16918
SEXO_1
1    60606
0    56153
Name: SEXO_1, dtype: int64
2
NIVEL_RELACIONAMENTO_CREDITO01
0.111111    110419
1.000000      1533
0.000000      1324
0.222222       733
0.888889       652
0.777778       612
0.666667       4

0.000000    14990
0.998928      302
0.037255      282
0.964458      234
0.000253      217
0.963185      183
0.989980      169
0.967579      168
0.998417      161
0.991472      151
Name: CAD_DEMOGRAFICO_VAR_17, dtype: int64
25880
CAD_DEMOGRAFICO_VAR_19
0.000000    14990
0.000037      371
0.999998      348
0.999987      327
0.000063      320
0.999999      319
0.999996      288
0.002678      282
0.999995      278
0.999985      269
Name: CAD_DEMOGRAFICO_VAR_19, dtype: int64
17759
CAD_DEMOGRAFICO_VAR_21
0.000000    14990
0.017351      282
0.991670      234
0.634559      216
0.238135      206
0.364606      183
0.170363      169
0.100960      168
0.216848      159
0.505530      150
Name: CAD_DEMOGRAFICO_VAR_21, dtype: int64
31671
CAD_DEMOGRAFICO_VAR_22
0.000000    14990
0.031813      282
0.993852      234
0.583199      216
0.982244      207
0.009870      183
0.010279      169
0.008048      168
0.145849      159
0.079105      150
Name: CAD_DEMOGRAFICO_VAR_22, dtype: int64
31198
CAD_DEMOGRAFICO

1.000000    1987
0.014348     232
0.020555     216
0.000153     206
0.058305     183
0.056720     169
0.310845     168
0.043373     159
0.195154     150
0.019281     132
Name: MENOR_DIST_ENDERECO_OFICINAS, dtype: int64
53260
MENOR_DIST_ENDERECO_LAVA_RAPIDO
 1.000000e+00    2113
-4.379933e-19    1165
 1.823984e-02     232
 3.700059e-02     216
 5.140202e-02     206
 6.619197e-02     183
 7.993909e-01     169
 1.973262e-01     168
 3.820978e-02     159
 1.362536e-01     150
Name: MENOR_DIST_ENDERECO_LAVA_RAPIDO, dtype: int64
52625
MENOR_DIST_ENDERECO_CEMITERIO
 1.000000e+00    2020
-1.371173e-18    1640
 3.017856e-02     232
 5.487558e-02     216
 5.523813e-02     206
 7.888535e-02     183
 5.301598e-02     169
 2.372356e-01     168
 3.655255e-02     159
 1.359753e-01     150
Name: MENOR_DIST_ENDERECO_CEMITERIO, dtype: int64
52373
MENOR_DIST_ENDERECO_IGREJA
 1.000000e+00    1802
-6.595748e-20    1763
 2.059771e-02     232
 6.125039e-02     216
 7.560011e-02     206
 4.109651e-02     183


0.000000    41697
0.007092     5466
0.014184     4078
0.021277     3647
0.028369     3180
0.035461     2564
0.049645     2463
0.042553     2152
1.000000     1824
0.070922     1711
Name: EXPOSICAO_ENDERECO_ALUGUEL_CARROS, dtype: int64
142
EXPOSICAO_ENDERECO_OFICINAS
0.000000    21444
0.001730     4456
0.003460     2448
0.005190     2248
0.006920     1872
0.008651     1529
0.010381     1300
1.000000     1279
0.019031     1274
0.013841     1072
Name: EXPOSICAO_ENDERECO_OFICINAS, dtype: int64
579
EXPOSICAO_ENDERECO_LAVA_RAPIDO
0.000000    34801
0.004762     5618
0.009524     3347
0.014286     2066
0.028571     1977
0.019048     1961
0.023810     1821
1.000000     1740
0.038095     1363
0.057143     1358
Name: EXPOSICAO_ENDERECO_LAVA_RAPIDO, dtype: int64
211
EXPOSICAO_ENDERECO_CEMITERIO
0.000000    33860
0.030303    14835
0.060606    12515
0.090909     9139
0.121212     7404
0.151515     5448
0.181818     4331
0.212121     3439
0.272727     2816
0.242424     2706
Name: EXPOSICAO_ENDERECO_CE

0    73619
1    43140
Name: CEP1_4, dtype: int64
2
CEP1_5
0    75626
1    41133
Name: CEP1_5, dtype: int64
2
CEP2_1
1    92290
0    24469
Name: CEP2_1, dtype: int64
2
CEP2_2
1    68773
0    47986
Name: CEP2_2, dtype: int64
2
CEP2_3
0    62039
1    54720
Name: CEP2_3, dtype: int64
2
CEP2_4
0    68207
1    48552
Name: CEP2_4, dtype: int64
2
CEP2_5
0    73441
1    43318
Name: CEP2_5, dtype: int64
2
CEP2_6
0    75089
1    41670
Name: CEP2_6, dtype: int64
2
CEP2_7
0    76798
1    39961
Name: CEP2_7, dtype: int64
2
CEP2_8
0    77259
1    39500
Name: CEP2_8, dtype: int64
2
CEP2_9
0    78507
1    38252
Name: CEP2_9, dtype: int64
2
CEP3_1
1    87393
0    29366
Name: CEP3_1, dtype: int64
2
CEP3_2
1    70758
0    46001
Name: CEP3_2, dtype: int64
2
CEP3_3
0    64494
1    52265
Name: CEP3_3, dtype: int64
2
CEP3_4
0    70271
1    46488
Name: CEP3_4, dtype: int64
2
CEP3_5
0    72258
1    44501
Name: CEP3_5, dtype: int64
2
CEP3_6
0    77876
1    38883
Name: CEP3_6, dtype: int64
2
CEP3_7
0    78308
1  

In [14]:
for column in category_columns:
    training_data[column] = training_data[column].astype('category')

In [15]:
training_data.dtypes.value_counts()

float64     144
category     80
int64        22
dtype: int64

In [16]:
confunsion_matrix = pandas.crosstab(training_data["SEXO_1"], training_data["IND_BOM_1_1"])
confunsion_matrix

IND_BOM_1_1,0,1
SEXO_1,,
0,19330,36823
1,20837,39769


In [17]:
from scipy.stats import chisquare
chisquare(confunsion_matrix)

Power_divergenceResult(statistic=array([ 56.54016979, 113.31360978]), pvalue=array([5.50619646e-14, 1.84208521e-26]))

# Feature Selection

In [18]:
features.corr().abs()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_5,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14
UF_1,1.000000,0.209093,0.048255,0.146628,0.118488,0.100570,0.087990,0.019386,0.007811,0.006435,...,0.025403,0.017901,0.002279,0.000400,0.041249,0.004074,0.004246,0.009868,0.022239,0.035315
UF_2,0.209093,1.000000,0.123264,0.247423,0.187327,0.171868,0.180898,0.012217,0.013103,0.014506,...,0.040997,0.006801,0.002159,0.035769,0.002821,0.023900,0.012583,0.021424,0.007369,0.029016
UF_3,0.048255,0.123264,1.000000,0.275167,0.269899,0.246315,0.216494,0.031256,0.003892,0.013546,...,0.011452,0.008139,0.009605,0.062716,0.011476,0.038354,0.017366,0.002350,0.018033,0.011395
UF_4,0.146628,0.247423,0.275167,1.000000,0.126074,0.126333,0.135131,0.020921,0.004924,0.041647,...,0.001024,0.005757,0.009747,0.021085,0.012738,0.019959,0.028406,0.010240,0.023383,0.047764
UF_5,0.118488,0.187327,0.269899,0.126074,1.000000,0.156790,0.121600,0.003239,0.006073,0.018759,...,0.000297,0.002524,0.060533,0.009569,0.000885,0.000713,0.001331,0.023599,0.026219,0.026264
UF_6,0.100570,0.171868,0.246315,0.126333,0.156790,1.000000,0.137971,0.026368,0.017433,0.028672,...,0.002480,0.023013,0.067999,0.022139,0.029033,0.065947,0.025372,0.000670,0.015740,0.003706
UF_7,0.087990,0.180898,0.216494,0.135131,0.121600,0.137971,1.000000,0.002974,0.022839,0.000691,...,0.044504,0.024435,0.005311,0.000662,0.006608,0.035075,0.001038,0.017101,0.034493,0.003431
IDADE,0.019386,0.012217,0.031256,0.020921,0.003239,0.026368,0.002974,1.000000,0.017405,0.029988,...,0.001120,0.004402,0.013037,0.001559,0.008037,0.004025,0.000544,0.002066,0.000616,0.006353
SEXO_1,0.007811,0.013103,0.003892,0.004924,0.006073,0.017433,0.022839,0.017405,1.000000,0.020646,...,0.000789,0.008388,0.000917,0.002623,0.002129,0.002683,0.001580,0.004715,0.002303,0.003675
NIVEL_RELACIONAMENTO_CREDITO01,0.006435,0.014506,0.013546,0.041647,0.018759,0.028672,0.000691,0.029988,0.020646,1.000000,...,0.002201,0.004603,0.001246,0.002234,0.003535,0.001846,0.000003,0.000372,0.004002,0.005085


In [19]:
training_data.head(10)

,id,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,33220,1,1,1,0,0,0,0,0.217846,0,...,0,0,1,1,0,0,0,0,0,1
1,164123,0,0,1,1,0,1,0,0.750400,0,...,0,1,0,1,1,0,0,0,0,1
2,340086,1,0,0,0,1,1,0,0.074953,0,...,0,0,0,0,0,1,1,1,1,0
3,237182,1,1,1,0,0,0,0,0.355855,0,...,0,1,1,1,0,0,0,1,1,0
4,335250,1,0,1,0,0,1,0,0.930834,1,...,0,1,0,0,0,1,0,0,1,0
5,149584,1,1,1,0,0,0,0,0.678045,0,...,1,0,0,1,0,0,1,0,1,0
6,71560,1,1,0,0,0,0,1,0.485231,1,...,1,0,1,0,1,0,0,0,1,0
7,118664,0,1,1,0,1,0,0,0.654419,1,...,0,0,0,0,0,1,1,1,1,0
8,19053,1,1,0,1,0,0,0,0.358808,1,...,0,0,0,0,1,1,0,1,1,0
9,368546,1,1,1,0,0,0,0,0.132485,1,...,1,1,1,1,0,0,0,1,1,0


In [20]:
corr_matrix = features.corr().abs()

In [21]:
values = corr_matrix[corr_matrix > 0.95].count()

In [22]:
values[values > 1]

FLAG_BOLSA_FAMILIA_1        2
RENDA_VIZINHANCA_1          2
RENDA_VIZINHANCA_4          2
FLAG_PROGRAMAS_SOCIAIS_1    2
dtype: int64

In [23]:
corr_matrix[corr_matrix["RENDA_VIZINHANCA_1"] > 0.90]

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_5,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14
RENDA_VIZINHANCA_1,0.039497,0.126106,0.095179,0.120255,0.040225,0.021913,0.094908,0.056833,0.011343,0.004249,...,0.014022,0.030272,0.002249,0.002028,0.003034,0.005432,0.003514,0.008236,0.007406,0.022686
RENDA_VIZINHANCA_4,0.042773,0.123006,0.096118,0.121031,0.041934,0.020677,0.093601,0.053994,0.009825,0.003914,...,0.014699,0.027470,0.003278,0.000880,0.003880,0.005072,0.002067,0.006980,0.006323,0.020939


RENDA_VIZINHANCA_1 e  RENDA_VIZINHANCA_4 possuem alta correlação e FLAG_BOLSA_FAMILIA_1 e FLAG_PROGRAMAS_SOCIAIS_1
também. Logo, vou ficar com somente duas das 4.
Contudo, este teste é para apenas para variáveis correlacionadas linearmente, existem testes melhores para as variáveis categóricas.

In [24]:
features = features.drop(["RENDA_VIZINHANCA_1", "FLAG_BOLSA_FAMILIA_1"], axis="columns")

In [25]:
features.shape

(116759, 241)

In [26]:
features.drop_duplicates(inplace=True)
features.shape

(116732, 241)

In [27]:
training_data_model = pandas.concat([features, labels], axis=1)

In [28]:
training_data_model.head(10)

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.217846,0.0,0.111111,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.750400,0.0,0.111111,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.074953,0.0,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.355855,0.0,0.111111,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.930834,1.0,0.111111,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
5,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.678045,0.0,0.111111,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
6,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.485231,1.0,0.111111,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
7,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.654419,1.0,0.111111,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
8,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.358808,1.0,0.111111,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1
9,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.132485,1.0,0.111111,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1


In [29]:
labels.value_counts()

1    76592
0    40167
Name: IND_BOM_1_1, dtype: int64

# Feature Engineering

In this module we're trying to build the feature that we see will be more useful in order to learn about
the class we need to predict.

# Model Tranining

In [30]:
features = training_data.drop(["IND_BOM_1_1", "IND_BOM_1_2", "id"], axis=1)

In [31]:
features.head(10)

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_5,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14
0,1,1,1,0,0,0,0,0.217846,0,0.111111,...,0,1,0,0,1,1,0,0,0,0
1,0,0,1,1,0,1,0,0.750400,0,0.111111,...,0,0,0,1,0,1,1,0,0,0
2,1,0,0,0,1,1,0,0.074953,0,0.111111,...,0,0,0,0,0,0,0,1,1,1
3,1,1,1,0,0,0,0,0.355855,0,0.111111,...,0,0,0,1,1,1,0,0,0,1
4,1,0,1,0,0,1,0,0.930834,1,0.111111,...,1,1,0,1,0,0,0,1,0,0
5,1,1,1,0,0,0,0,0.678045,0,0.111111,...,1,0,1,0,0,1,0,0,1,0
6,1,1,0,0,0,0,1,0.485231,1,0.111111,...,0,1,1,0,1,0,1,0,0,0
7,0,1,1,0,1,0,0,0.654419,1,0.111111,...,0,1,0,0,0,0,0,1,1,1
8,1,1,0,1,0,0,0,0.358808,1,0.111111,...,1,1,0,0,0,0,1,1,0,1
9,1,1,1,0,0,0,0,0.132485,1,0.111111,...,0,0,1,1,1,1,0,0,0,1


In [32]:
labels = training_data["IND_BOM_1_1"]

In [33]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler

In [34]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=1/4, 
                                                    random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/3, 
                                                  random_state=42, stratify=y_train)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [35]:
input_dimension = X_train.shape[1]
input_dimension

243

In [36]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=20,
                              verbose=0, mode='auto')

In [37]:
classifier_1 = Sequential()
classifier_1.add(Dense(16, activation='tanh', input_dim=input_dimension))
classifier_1.add(Dense(16, activation='relu', input_dim=input_dimension))
classifier_1.add(Dense(1, activation='sigmoid'))

classifier_1.compile(optimizer='adam', loss='mean_squared_error', metrics=["accuracy"])

In [38]:
classifier_2 = Sequential()
classifier_2.add(Dense(16, activation='tanh', input_dim=input_dimension))
classifier_2.add(Dense(16, activation='tanh', input_dim=input_dimension))
classifier_2.add(Dense(8, activation='relu', input_dim=input_dimension/2))
classifier_2.add(Dense(1, activation='sigmoid'))

classifier_2.compile(optimizer='adam', loss='mean_squared_error', metrics=["accuracy"])

In [39]:
classifier_3 = Sequential()
classifier_3.add(Dense(16, activation='relu', input_dim=input_dimension))
classifier_3.add(Dense(8, activation='relu', input_dim=input_dimension))
classifier_3.add(Dense(1, activation='sigmoid'))

classifier_3.compile(optimizer='adam', loss='mean_squared_error', metrics=["accuracy"])

I use as_matrix because Keras expects a Numpy array instead of a dataframe.

In [40]:
classifier_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                3904      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 4,193
Trainable params: 4,193
Non-trainable params: 0
_________________________________________________________________


In [41]:
classifier_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                3904      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 4,321
Trainable params: 4,321
Non-trainable params: 0
_________________________________________________________________


In [42]:
classifier_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 16)                3904      
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 9         
Total params: 4,049
Trainable params: 4,049
Non-trainable params: 0
_________________________________________________________________


In [43]:
model = classifier_1.fit(X_train.as_matrix(), y_train.as_matrix(),epochs=500, callbacks=[early_stopping], validation_split=0.15)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Train on 49622 samples, validate on 8757 samples
Epoch 1/500
49622/49622 [==============================] - 6s 124us/step - loss: 0.2145 - acc: 0.6632 - val_loss: 0.2072 - val_acc: 0.6735
Epoch 2/500
49622/49622 [==============================] - 7s 134us/step - loss: 0.2083 - acc: 0.6707 - val_loss: 0.2057 - val_acc: 0.6763
Epoch 3/500
49622/49622 [==============================] - 6s 124us/step - loss: 0.2066 - acc: 0.6737 - val_loss: 0.2047 - val_acc: 0.6788
Epoch 4/500
49622/49622 [==============================] - 5s 110us/step - loss: 0.2050 - acc: 0.6768 - val_loss: 0.2047 - val_acc: 0.6752
Epoch 5/500
49622/49622 [==============================] - 5s 105us/step - loss: 0.2039 - acc: 0.6798 - val_loss: 0.2041 - val_acc: 0.6780
Epoch 6/500
49622/49622 [==============================] - 8s 160us/step - loss: 0.2030 - acc: 0.6832 - val_loss: 0.2039 - val_acc: 0.6753
Epoch 7/500
49622/49622 [==============================] - 7s 144us/step - loss: 0.2021 - acc: 0.6834 - val_loss: 0.2

In [45]:
model = classifier_3.fit(X_train.as_matrix(), y_train.as_matrix(),epochs=500, callbacks=[early_stopping], validation_split=0.15)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Train on 49622 samples, validate on 8757 samples
Epoch 1/500
49622/49622 [==============================] - 8s 164us/step - loss: 0.2154 - acc: 0.6602 - val_loss: 0.2080 - val_acc: 0.6726
Epoch 2/500
49622/49622 [==============================] - 7s 136us/step - loss: 0.2080 - acc: 0.6707 - val_loss: 0.2056 - val_acc: 0.6783
Epoch 3/500
49622/49622 [==============================] - 8s 167us/step - loss: 0.2059 - acc: 0.6748 - val_loss: 0.2071 - val_acc: 0.6758
Epoch 4/500
49622/49622 [==============================] - 6s 122us/step - loss: 0.2049 - acc: 0.6765 - val_loss: 0.2038 - val_acc: 0.6763
Epoch 5/500
49622/49622 [==============================] - 7s 139us/step - loss: 0.2039 - acc: 0.6809 - val_loss: 0.2052 - val_acc: 0.6750
Epoch 6/500
49622/49622 [==============================] - 7s 132us/step - loss: 0.2030 - acc: 0.6818 - val_loss: 0.2060 - val_acc: 0.6684
Epoch 7/500
49622/49622 [==============================] - 6s 130us/step - loss: 0.2026 - acc: 0.6838 - val_loss: 0.2

In [44]:
model = classifier_2.fit(X_train.as_matrix(), y_train.as_matrix(),epochs=500, callbacks=[early_stopping], validation_split=0.15)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Train on 49622 samples, validate on 8757 samples
Epoch 1/500
49622/49622 [==============================] - 6s 125us/step - loss: 0.2142 - acc: 0.6627 - val_loss: 0.2063 - val_acc: 0.6773
Epoch 2/500
49622/49622 [==============================] - 6s 114us/step - loss: 0.2075 - acc: 0.6710 - val_loss: 0.2053 - val_acc: 0.6817
Epoch 3/500
49622/49622 [==============================] - 6s 118us/step - loss: 0.2057 - acc: 0.6737 - val_loss: 0.2077 - val_acc: 0.6815
Epoch 4/500
49622/49622 [==============================] - 6s 127us/step - loss: 0.2042 - acc: 0.6785 - val_loss: 0.2043 - val_acc: 0.6824
Epoch 5/500
49622/49622 [==============================] - 8s 156us/step - loss: 0.2034 - acc: 0.6784 - val_loss: 0.2042 - val_acc: 0.6845
Epoch 6/500
49622/49622 [==============================] - 6s 125us/step - loss: 0.2024 - acc: 0.6815 - val_loss: 0.2042 - val_acc: 0.6759
Epoch 7/500
49622/49622 [==============================] - 6s 119us/step - loss: 0.2014 - acc: 0.6844 - val_loss: 0.2

In [46]:
from keras.layers import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense
from keras.layers.merge import concatenate

In [47]:
inputs = Input(shape=(243,))

x1 = Dense(16, activation="tanh")(inputs)
x1 = Dense(16, activation="relu")(x1)

x2 = Dense(16, activation="tanh")(inputs)
x2 = Dense(8, activation="relu")(x2)

x3 = Dense(16, activation="tanh")(inputs)
x3 = Dense(16, activation="tanh")(x3)
x3 = Dense(8, activation="relu")(x3)

x4 = concatenate([x1,x2,x3])

prediction = Dense(1, activation="sigmoid")(x4)

voting_classifier = Model(inputs=inputs, outputs= prediction)
voting_classifier.compile(optimizer='adam', loss='mean_squared_error', metrics=["accuracy"])

In [53]:
early_stopping_voting = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=0, mode='auto')

In [54]:
voting_classifier.fit(X_train.as_matrix(), y_train.as_matrix(),epochs=500, callbacks=[early_stopping_voting], validation_split=0.15)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Train on 49622 samples, validate on 8757 samples
Epoch 1/500
49622/49622 [==============================] - 6s 127us/step - loss: 0.1666 - acc: 0.7577 - val_loss: 0.2292 - val_acc: 0.6459
Epoch 2/500
49622/49622 [==============================] - 7s 136us/step - loss: 0.1654 - acc: 0.7612 - val_loss: 0.2301 - val_acc: 0.6517
Epoch 3/500
49622/49622 [==============================] - 7s 140us/step - loss: 0.1639 - acc: 0.7650 - val_loss: 0.2323 - val_acc: 0.6433
Epoch 4/500
49622/49622 [==============================] - 6s 126us/step - loss: 0.1629 - acc: 0.7669 - val_loss: 0.2308 - val_acc: 0.6599
Epoch 5/500
49622/49622 [==============================] - 7s 131us/step - loss: 0.1617 - acc: 0.7688 - val_loss: 0.2331 - val_acc: 0.6493
Epoch 6/500
49622/49622 [==============================] - 7s 136us/step - loss: 0.1604 - acc: 0.7738 - val_loss: 0.2350 - val_acc: 0.6433
Epoch 7/500
49622/49622 [==============================] - 7s 135us/step - loss: 0.1592 - acc: 0.7733 - val_loss: 0.2

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier

In [ ]:
voting_classifier = VotingClassifier([('classifier_1', classifier_1), ('classifier_2', classifier_2), ('classifier_3',classifier_3)], voting='soft')

In [ ]:
voting_classifier.fit(X_train, y_train)

# Random Forest

In [ ]:
rf_clf = RandomForestClassifier()  # Modifique aqui os hyperparâmetros
rf_clf.fit(X_train, y_train)


In [ ]:
rf_pred_class = rf_clf.predict(X_val)
rf_pred_scores = rf_clf.predict_proba(X_val)[:, 1]
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_val, rf_pred_class, rf_pred_scores)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

In [ ]:
rf_pred_test_class = rf_clf.predict(X_test)
rf_pred_test_scores = rf_clf.predict_proba(X_test)[:, 1]
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, rf_pred_test_class, rf_pred_test_scores)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

# XGBoost

## Parameter Selection

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
# We will use grid search to improve our search for parameters using Cross Validation
number_estimators = [30, 60, 120, 200]
loss_function = ["deviance", "exponential"]
min_samples_leaf = [1, 0.05]
sub_samples = [1.0, 0.8, 0.6]
max_features = ["log2", "sqrt", "auto"]
xgboost_classifier = GridSearchCV(estimator=GradientBoostingClassifier(), 
                          param_grid=dict(
                              n_estimators=number_estimators,
                              max_features=max_features,
                              subsample=sub_samples,
                          min_samples_leaf=min_samples_leaf,
                          loss=loss_function), n_jobs=-1)

In [ ]:
xgboost_classifier.fit(X_train, y_train)

In [ ]:
# as I said in the documentation, GridSeach uses a stratified 3-fold cross validation because a Classifier was passed
# instead of a regressor

classifier.best_params_

In [ ]:
# For in ensemble classifiers
classifier = xgboost_classifier

# Ensemble classifiers (Voting)

## Random Forest and XGBoost together

In [ ]:
classifier = VotingClassifier([('xgboost', xgboost_classifier), ('randomforest', rf_clf)], voting='soft')

In [ ]:
classifier.fit(X_train, y_train)

# Evaluation

In [55]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score

In [ ]:
y_train_pred = classifier.predict(X_train.as_matrix()).ravel()
y_test_pred = classifier.predict(X_test.as_matrix()).ravel()

é bom prestar atenção se os valores estão próximo, caso contrário, existe uma boa indicação de que houve
overfitting e o modelo não consegue generalizar tão bem.

In [ ]:
print("Mean Square error in train: {:0.1f}".format(mse(y_train, y_train_pred)))
print("Mean Square error in test: {:0.1f}".format(mse(y_test, y_test_pred)))

In [60]:
def compute_performance_metrics(y, y_pred_class, y_pred_scores=None):
    accuracy = accuracy_score(y, y_pred_class)
    recall = recall_score(y, y_pred_class)
    precision = precision_score(y, y_pred_class)
    f1 = f1_score(y, y_pred_class)
    performance_metrics = (accuracy, recall, precision, f1)
    if y_pred_scores is not None:
        auroc = roc_auc_score(y, y_pred_scores)
        aupr = average_precision_score(y, y_pred_scores)
        performance_metrics = performance_metrics + (auroc, aupr)
    return performance_metrics

In [61]:
def print_metrics_summary(accuracy, recall, precision, f1, auroc=None, aupr=None):
    print()
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1))
    if auroc is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=auroc))
    if aupr is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUPR:", value=aupr))

In [ ]:
# This returns an array for the probabilities of being each class, so it depends what the focus will be
# in our case, the focus is at class 1
y_test_pred_prob = classifier.predict_proba(X_test.as_matrix())[:, 0]

In [ ]:
y_test_pred.shape

In [ ]:
y_test.shape

In [ ]:
y_test_pred_prob.shape

In [ ]:
print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

# Evaluate for Kaggle

In [ ]:
kaggle_test_data = pandas.read_csv("real_test_set.csv")

In [ ]:
kaggle_test_data.shape
features_kaggle = kaggle_test_data.drop(["id"], axis=1)
features_kaggle.head(10)

In [ ]:
rf_pred_test_class = rf_clf.predict(features_kaggle)
rf_pred_test_scores = rf_clf.predict_proba(features_kaggle)[:, 1]

In [ ]:
rf_pred_test_class.size

In [ ]:
rf_pred_test_class

Se ligar que na hora que cria o csv, na primeira linha (a linha do header), ele coloca ",0", tem que substituir para "id,IND_BOM_1_1"

In [ ]:
df = pandas.DataFrame(data=rf_pred_test_class)
df.to_csv('test.csv', mode='a', index=True)

In [ ]:
# For in ensemble classifiers
classifier = xgboost_classifier

# RESULTS LOG

## XGBoost - 120 estimators

In [ ]:

print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

In [ ]:

print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

## XGBoost - 2nd configuration

In [ ]:
# parameters
number_estimators = [30, 60, 120, 200]
loss_function = ["deviance", "exponential"]
min_samples_leaf = [1, 0.05]
sub_samples = [1.0, 0.8, 0.6]
max_features = ["log2", "sqrt", "auto"]

In [ ]:
classifier.best_params_

In [ ]:

print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

# Ensemble Of XBGoost and RandomForest

# MLPs and Ensemble MLPS

In [65]:
y_train_pred = classifier_1.predict(X_train.as_matrix()).ravel()
y_test_pred = classifier_1.predict(X_test.as_matrix()).ravel()

# This returns an array for the probabilities of being each class, so it depends what the focus will be
# in our case, the focus is at class 1
y_test_pred_prob = classifier_1.predict_proba(X_test.as_matrix())[:, 0]

print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  



Performance no conjunto de teste:

Accuracy:         0.6664
Recall:           0.8560
Precision:        0.7013
F1:               0.7710
AUROC:            0.6650
AUPR:             0.7853


In [66]:
y_train_pred = classifier_2.predict(X_train.as_matrix()).ravel()
y_test_pred = classifier_2.predict(X_test.as_matrix()).ravel()

# This returns an array for the probabilities of being each class, so it depends what the focus will be
# in our case, the focus is at class 1
y_test_pred_prob = classifier_2.predict_proba(X_test.as_matrix())[:, 0]

print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  



Performance no conjunto de teste:

Accuracy:         0.6660
Recall:           0.8400
Precision:        0.7064
F1:               0.7674
AUROC:            0.6698
AUPR:             0.7878


In [67]:
y_train_pred = classifier_3.predict(X_train.as_matrix()).ravel()
y_test_pred = classifier_3.predict(X_test.as_matrix()).ravel()

# This returns an array for the probabilities of being each class, so it depends what the focus will be
# in our case, the focus is at class 1
y_test_pred_prob = classifier_3.predict_proba(X_test.as_matrix())[:, 0]

print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  



Performance no conjunto de teste:

Accuracy:         0.6684
Recall:           0.8315
Precision:        0.7116
F1:               0.7669
AUROC:            0.6739
AUPR:             0.7916


In [68]:
y_train_pred = voting_classifier.predict(X_train.as_matrix()).ravel()
y_test_pred = voting_classifier.predict(X_test.as_matrix()).ravel()

# This returns an array for the probabilities of being each class, so it depends what the focus will be
# in our case, the focus is at class 1
y_test_pred_prob = classifier_1.predict_proba(X_test.as_matrix())[:, 0]

print('\nPerformance no conjunto de teste:')
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(y_test, y_test_pred.round(), y_test_pred_prob)
print_metrics_summary(accuracy, recall, precision, f1, auroc, aupr)

C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\danil\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  



Performance no conjunto de teste:

Accuracy:         0.6493
Recall:           0.7997
Precision:        0.7051
F1:               0.7495
AUROC:            0.6650
AUPR:             0.7853
